#  Sashimi plot

In [ ]:
#cd ~/Work/leaf_cutter2/ROSMAP_DLPFC/output/sashimi_plots/19samples/plots
#ln -s ../../test/test_data_to_Chao/sashimi_ingredients.py  ./
#ln -s ../../test/test_data_to_Chao/template.ini  ./
#ln -s ../bam_files/*bw ./
    sos run ~/codes/xqtl-pipeline/code/molecular_phenotypes/sashimi_plot.ipynb sashimi_plot\
    --recipe /mnt/vast/hpc/csg/rf2872/Work/leaf_cutter2/ROSMAP_DLPFC/output/sashimi_plots/19samples/snps/finemap_clusemp_overlap.txt \
    --individuals_list /mnt/vast/hpc/csg/rf2872/Work/leaf_cutter2/ROSMAP_DLPFC/output/sashimi_plots/test/test_data_to_Chao/individuals.txt \
    -J 200 -c ~/test/csg.yml -q csg -s build --job_size 10 --mem 15G --numThreads 8 \
    --cwd output_fmp_clusemp

In [ ]:
sos run ~/codes/xqtl-pipeline/code/molecular_phenotypes/sashimi_plot.ipynb sashimi_plot\
    --recipe /mnt/vast/hpc/csg/rf2872/Work/leaf_cutter2/ROSMAP_DLPFC/output/sashimi_plots/19samples/snps/finemap_clusemp_overlap_expand.txt \
    --individuals_list /mnt/vast/hpc/csg/rf2872/Work/leaf_cutter2/ROSMAP_DLPFC/output/sashimi_plots/test/test_data_to_Chao/individuals.txt \
    -J 200 -c ~/test/csg.yml -q csg -s build --job_size 10 --mem 15G --numThreads 8 \
    --cwd output_fmp_clusemp_expand

In [ ]:
sos run ~/codes/xqtl-pipeline/code/molecular_phenotypes/sashimi_plot.ipynb sashimi_plot\
    --recipe /mnt/vast/hpc/csg/rf2872/Work/leaf_cutter2/ROSMAP_DLPFC/output/sashimi_plots/19samples/snps/finemap_clusemp_AD_overlap.txt \
    --individuals_list /mnt/vast/hpc/csg/rf2872/Work/leaf_cutter2/ROSMAP_DLPFC/output/sashimi_plots/test/test_data_to_Chao/individuals.txt \
    -J 200 -c ~/test/csg.yml -q csg -s build --job_size 10 --mem 15G --numThreads 8 \
    --cwd output_fmp_clusemp_AD

In [ ]:
sos run ~/codes/xqtl-pipeline/code/molecular_phenotypes/sashimi_plot.ipynb sashimi_plot\
    --recipe /mnt/vast/hpc/csg/rf2872/Work/leaf_cutter2/ROSMAP_DLPFC/output/sashimi_plots/19samples/snps/finemap_clusemp_AD_overlap_expand.txt \
    --individuals_list /mnt/vast/hpc/csg/rf2872/Work/leaf_cutter2/ROSMAP_DLPFC/output/sashimi_plots/test/test_data_to_Chao/individuals.txt \
    -J 200 -c ~/test/csg.yml -q csg -s build --job_size 10 --mem 15G --numThreads 8 \
    --cwd output_fmp_clusemp_AD_expand

In [6]:
[global]
# The output directory for generated files.
parameter: cwd = path("output")
# Sample meta data list
parameter: recipe = path
#parameter: bam_files = path
parameter: individuals_list = path
# Raw data directory, default to the same directory as sample list
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Memory for Java virtual mechine (`picard`)
parameter: java_mem = "6G"
# Number of threads
parameter: numThreads = 8
# Software container option
parameter: container = ""
parameter: entrypoint={('micromamba run -n' + ' ' + container.split('/')[-1][:-4]) if container.endswith('.sif') else f''}
import pandas as pd
recipe = pd.read_csv(recipe,sep = "\t")
recipe = recipe.to_dict("records")

In [ ]:
[sashimi_plot_1]
input: individuals_list, for_each="recipe"
output: f'{cwd:a}/{_recipe["clus_id"]}/chr{_recipe["chromosome"]}_{_recipe["position"]}/sashimi_{_recipe["clus_id"]}_chr{_recipe["chromosome"]}_{_recipe["position"]}_plot.pdf'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
bash: expand= "$[ ]", stdout = f"{_output[0]:n}.stdout", stderr = f"{_output[0]:n}.stderr", container = container, entrypoint = entrypoint
        data_pre_region="$[_recipe["data_pre_region"]]"
        top_snp="$[_recipe["top_snp"]]"

        plot_region="$[_recipe["plot_region"]]"
        counts="$[_recipe["counts"]]"

        vcf="$[_recipe["vcf"]]"
        sample_id_list="$[_input]"

        clu="$[_recipe["clus_id"]]"
        #clu=$(echo $counts | sed -n 's/[^_]*_\(.*\)\.txt/\1/p')
        #out_pdf=sashimi_"$[_recipe["clus_id"]]"_$[_recipe["variants"]].pdf
        
        cd $[cwd:a]/$[_recipe["clus_id"]]/chr$[_recipe["chromosome"]]_$[_recipe["position"]]/
        module load Bedtools/2.30
        # data prep
        ../../../sashimi_ingredients.py \
        -V $vcf \
        -B <(realpath ../../../*.bw) \
        -C $counts \
        -P $sample_id_list \
        -T ../../../template.ini \
        -S $top_snp \
        -R $data_pre_region \
        -E 1000 \
        -O ./plot \
        && echo "data prep done" \
        && echo "plot sashimi: $plot_region" \
        && ~/miniconda3/envs/pygenometracks2/bin/pyGenomeTracks \
                --tracks plot.ini \
                --region  $plot_region \
                --title "QTL:${plot_region}|TOP:${top_snp}|$clu"\
                --width 30 \
                --trackLabelFraction .1 --fontSize 10 \
                -out $[_output] \
        && echo "plot done"

In [ ]:
[sashimi_plot_2]
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
bash: expand= "$[ ]", stdout = f"{_output[0]:n}.stdout", stderr = f"{_output[0]:n}.stderr", container = container, entrypoint = entrypoint
        mkdir -p $[cwd:a]/plots_pdf && cd $[cwd:a]/
        ln -s ../*/*/*pdf ./